In [1]:
from huggingface_hub import notebook_login

notebook_login()  # same as before 

In [26]:
from datasets import load_dataset

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

In [28]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments, EvalPrediction
import numpy as np
import evaluate
from sklearn.model_selection import train_test_split

# Load your dataset
dataset = load_dataset('csv', data_files='dataset.csv', encoding='ISO-8859-1')['train']

Generating train split: 0 examples [00:00, ? examples/s]

In [29]:
dataset

Dataset({
    features: ['title', 'labels'],
    num_rows: 181688
})

In [30]:
final_data = dataset.train_test_split(test_size=0.25)

In [31]:
final_data

DatasetDict({
    train: Dataset({
        features: ['title', 'labels'],
        num_rows: 136266
    })
    test: Dataset({
        features: ['title', 'labels'],
        num_rows: 45422
    })
})

In [32]:
from transformers import TFAutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = TFAutoModel.from_pretrained('distilbert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [33]:
def preprocess_function(examples):
    return tokenizer(examples["title"], truncation=True)

In [34]:
tokenized_data = final_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/136266 [00:00<?, ? examples/s]

Map:   0%|          | 0/45422 [00:00<?, ? examples/s]

In [35]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['title', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 136266
    })
    test: Dataset({
        features: ['title', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 45422
    })
})

In [36]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [37]:
import evaluate

accuracy = evaluate.load("accuracy")

In [38]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [39]:
id2label = {0: "informational", 1: "transactional",2:"navigational_informational",3:"navigational_transactional",4:"transactional_download"}
label2id = {"informational": 0, "transactional": 1, "navigational_informational": 2, "navigational_transactional": 3, "transactional_download":4}

In [40]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 3
batches_per_epoch = len(tokenized_data["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [43]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=5, id2label=id2label, label2id=label2id
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [44]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_data["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_data["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [45]:
import tensorflow as tf

model.compile(optimizer=optimizer)  # No loss argument!

In [46]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [47]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="Mukalingam0813/intent-classifier",
    tokenizer=tokenizer,
)

C:\Users\mukal\anaconda3\envs\intent\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
C:\Users\mukal\OneDrive\Documents\Intent\Mukalingam0813\intent-classifier is already a clone of https://huggingface.co/Mukalingam0813/intent-classifier. Make sure you pull the latest changes with `repo.git_pull()`.


In [48]:
callbacks = [metric_callback, push_to_hub_callback]

In [49]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=callbacks)

Epoch 1/3
8516/8516 [==============================] - 7798s 914ms/step - loss: 0.1562 - val_loss: 0.1044 - accuracy: 0.9679
Epoch 2/3
8516/8516 [==============================] - 7837s 920ms/step - loss: 0.0654 - val_loss: 0.0950 - accuracy: 0.9723
Epoch 3/3
8516/8516 [==============================] - 7915s 929ms/step - loss: 0.0309 - val_loss: 0.1017 - accuracy: 0.9726


In [50]:
text = "Allocating a $3 million innovation fund to spearhead the next generation of artificial intelligence technologies."

In [51]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Mukalingam0813/intent-classifier")
inputs = tokenizer(text, return_tensors="tf")

In [52]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("Mukalingam0813/intent-classifier")
logits = model(**inputs).logits

Some layers from the model checkpoint at Mukalingam0813/intent-classifier were not used when initializing TFDistilBertForSequenceClassification: ['dropout_97']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at Mukalingam0813/intent-classifier and are newly initialized: ['dropout_117']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [53]:
predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
model.config.id2label[predicted_class_id]

'informational'